In [226]:
import pandas as pd
import numpy as np
import ast
import matplotlib.pyplot as plt

In [227]:
df = pd.read_csv('전선_1202_수동전처리.csv',encoding='cp949')

In [228]:
df.shape

(3405, 74)

In [229]:
df[df['수강신청유의사항'] == '#NAME?']

,연도,과목번호,교과목명,학점,시간,개설학년,개설학과,강의종류,강의유형,전체학년 제한인원,...,교재,과제내용,과제수,주별강의계획서,중간고사,기말고사,담당교수,학수번호,캡스톤,NCS


## 파생변수 정의
- 선수과목 존재 여부
- 조별과제 존재 여부
- 지필고사 대체 여부
- 수강대상 보호 여부
- 유학생, 교환학생 수강제한 여부
- 1교시 여부
- 월요일 여부
- 금요일 여부
- 이론강의팀플존재여부

In [230]:
df.rename(columns={'과제비율':'평가비율'},inplace=True)

In [231]:
# 딕셔너리로 저장된 데이터를 str에서 딕셔너리로 변환
def convert_to_dictionary(x):
    try:
        return ast.literal_eval(x)
    except Exception:
        return x

In [232]:
df['평가비율'] = df['평가비율'].apply(convert_to_dictionary)
df['교재'] = df['교재'].apply(convert_to_dictionary)
df['과제내용'] = df['과제내용'].apply(convert_to_dictionary)
df['주별강의계획서'] = df['주별강의계획서'].apply(convert_to_dictionary)
df['중간고사'] = df['중간고사'].apply(convert_to_dictionary)
df['기말고사'] = df['기말고사'].apply(convert_to_dictionary)

In [233]:
# 결측치 처리
df['비고'] = df['비고'].fillna('')
df['수강신청유의사항'] = df['수강신청유의사항'].fillna('')

In [234]:
df.columns

Index(['연도', '과목번호', '교과목명', '학점', '시간', '개설학년', '개설학과', '강의종류', '강의유형',
       '전체학년 제한인원', '전체학년 수강바구니', '전체학년 경쟁률', '4학년 제한인원', '4학년 수강바구니',
       '4학년 경쟁률', '3학년 제한인원', '3학년 수강바구니', '3학년 경쟁률', '2학년 제한인원', '2학년 수강바구니',
       '2학년 경쟁률', '1학년 제한인원', '1학년 수강바구니', '1학년 경쟁률', '수업요일', '건물명',
       '일주일 중 수업횟수', '수업시간대', '층', '분반 개수', '패스과목', '원어강의', '절대평가 유무',
       '점심시간 포함 개수', '고학년', '계열', '1학년 남학생 수', '1학년 여학생 수', '1학년 총 학생 수',
       '2학년 남학생 수', '2학년 여학생 수', '2학년 총 학생 수', '3학년 남학생 수', '3학년 여학생 수',
       '3학년 총 학생 수', '4학년 남학생 수', '4학년 여학생 수', '4학년 총 학생 수', '학과 총 학생 수',
       '학과 총 남학생 수', '학과 총 여학생 수', '1학년 남녀 비율', '2학년 남녀 비율', '3학년 남녀 비율',
       '4학년 남녀 비율', '학과 남녀 비율', '개설학년 경쟁률', '개설학년 수강바구니', '개설학년 제한인원', '단과대학',
       '원어여부', '비고', '수강신청유의사항', '평가비율', '교재', '과제내용', '과제수', '주별강의계획서',
       '중간고사', '기말고사', '담당교수', '학수번호', '캡스톤', 'NCS'],
      dtype='object')

### 중복된 컬럼 확인

In [235]:
def check_foreign(df):
    # 첫 번째 조건
    condition1 = (df['원어강의'] != 1.0) & (df['원어여부'].isin(['영어', '중국어']))
    mismatches1 = df[condition1]
    for idx, row in mismatches1.iterrows():
        print(f"{idx}번째 행: {row['원어강의']} vs. {row['원어여부']}")

    # 두 번째 조건
    condition2 = (df['원어강의'] == 1.0) & (df['원어여부'] == '')
    mismatches2 = df[condition2]
    for idx, row in mismatches2.iterrows():
        print(f"{idx}번째 행: {row['원어강의']} vs. {row['원어여부']}")

In [236]:
check_foreign(df)

In [237]:
df.drop(columns=['원어여부'],inplace=True)

### 결측치 확인

In [238]:
series = pd.Series(df.isnull().sum())
series[series > 0]

수업요일        15
건물명         15
수업시간대       15
층           15
평가비율         2
교재         838
과제내용       295
주별강의계획서      8
중간고사         8
기말고사         8
dtype: int64

In [239]:
df[df['층'].isna()][['연도','과목번호','교과목명','강의종류','강의유형']]

,연도,과목번호,교과목명,강의종류,강의유형
106,2022,3703,디지털그래픽제작및교육활용,e-러닝,이론+실습
173,2022,3397,제어공학,e-러닝,이론
823,2022,3314,무선통신공학,e-러닝,이론
1209,2023,3982,환경경영과정보관리,e-러닝,이론
1210,2023,3983,사회적책임경영과리스크관리,e-러닝,이론
1251,2023,3691,디지털그래픽제작및교육활용,e-러닝,이론+실습
1319,2023,3381,제어공학,e-러닝,이론
1362,2023,3428,미래자동차공학개론,e-러닝,이론
1363,2023,3429,전동화자동차공학,e-러닝,이론
2344,2024,3996,환경경영과정보관리,e-러닝,이론


In [240]:
df[['수업요일','건물명','수업시간대','층']] = df[['수업요일','건물명','수업시간대','층']].fillna('온라인')

### 선수과목 존재 여부

In [241]:
keywords = ['선수과목', '선이수', '이수요망', '이수자만', 'prerequisite', '듣지않은', 'C프로그래밍필수']
no_keywords = ['선수과목이필', '선수과목은필', '선수과목은없', 'keytheories', 'Noprerequisite']

df['선수과목존재여부'] = 0

def filter_text(raw_text):
    # 텍스트 내 공백 제거
    text = raw_text.replace(" ", "")
    # 키워드 포함 여부 확인
    contains_keyword = any(keyword in text for keyword in keywords)
    # 제외 키워드 포함 여부 확인
    contains_no_keyword = any(no_keyword in text for no_keyword in no_keywords)
    # 조건에 따라 T/F 반환
    return contains_keyword and not contains_no_keyword

# DataFrame에 두 컬럼에 대해 조건 적용
df['선수과목존재여부'] = df.apply(
    lambda row: 1 if (filter_text(row['비고']) or filter_text(row['수강신청유의사항'])) else 0, 
    axis=1
)

In [242]:
df.선수과목존재여부.value_counts()

선수과목존재여부
0    3235
1     170
Name: count, dtype: int64

### 조별과제 존재 여부

In [243]:
keywords = ['팀프로젝트', '조별', '팀플', '팀원', '팀별', 'team']
no_keywords = ['팀프로젝트는없','조별활동은없','조별과제는없','팀별활동은없','팀별과제는없',
               '팀프로젝트가없','조별활동이없','팀별활동이없','noteam']

df['조별과제존재여부'] = 0

def filter_text(raw_text):
    # 텍스트 내 공백 제거
    text = raw_text.replace(" ", "")
    # 키워드 포함 여부 확인
    contains_keyword = any(keyword in text for keyword in keywords)
    # 제외 키워드 포함 여부 확인
    contains_no_keyword = any(no_keyword in text for no_keyword in no_keywords)
    # 조건에 따라 T/F 반환
    return contains_keyword and not contains_no_keyword

# DataFrame에 두 컬럼에 대해 조건 적용
df['조별과제존재여부'] = df.apply(
    lambda row: 1 if (filter_text(row['비고']) or filter_text(row['수강신청유의사항'])) else 0, 
    axis=1
)

In [244]:
# filtered_rows = df[df['수강신청유의사항'].apply(filter_text)]
# for raw_text in filtered_rows['수강신청유의사항']:
#     print(raw_text)
#     print('-----------')

In [245]:
df.조별과제존재여부.value_counts()

조별과제존재여부
0    3179
1     226
Name: count, dtype: int64

### 지필고사 대체 여부

In [246]:
keywords1 = ['대체','발표','제출','프로젝트','포트폴리오','과제물','팀플','프레젠테이션','프리젠테이션',
             'project','presentation','assignment','Project','Presentation','Assignment','중간과제','기말과제']
keywords2 = ['과제대체','대체과제','과제물로대체']

df['지필고사대체여부'] = 0 

for i in range(len(df)):
    # 딕셔너리로 저장된 시험내용 데이터의 value 가져오기
    try:
        mid_exam = df.loc[i,'중간고사'].values()
        final_exam = df.loc[i,'기말고사'].values()
    except:
        pass

    # 딕셔너리로 저장된 과제내용 데이터의 value 가져오기
    try:
        homework = df.loc[i,'과제내용'].values()
    except:
        pass

    # 원소들을 하나의 문자열로 합친 후 공백을 제거하기
    mid_text = ''.join(mid_exam).replace(" ", "")
    final_text = ''.join(final_exam).replace(" ", "")
    text = mid_text + final_text    

    caution = df.loc[i,'수강신청유의사항'].replace(" ", "")
    remark = df.loc[i,'비고'].replace(" ", "")
    hw_text = ''.join(homework).replace(" ", "")
    
    # 시험 과제대체 여부 검색
    if '실습' not in df.loc[i,'교과목명'] or '실기' not in df.loc[i,'교과목명']:
        if any(word in text for word in keywords1):
            df.loc[i,'지필고사대체여부'] = 1
        elif any(word in caution for word in keywords2):
            df.loc[i,'지필고사대체여부'] = 1
        elif any(word in remark for word in keywords2):
            df.loc[i,'지필고사대체여부'] = 1
        elif any(word in hw_text for word in keywords2):
            df.loc[i,'지필고사대체여부'] = 1

In [247]:
df.지필고사대체여부.value_counts()

지필고사대체여부
0    2519
1     886
Name: count, dtype: int64

### 수강대상 보호 여부
- 원전공생에게만/우선적으로 수강신청 기회를 제공하는 경우
- 수강 대상 학년에게만/우선적으로 수강신청 기회를 제공하는 경우

In [248]:
df['수강대상보호'] = 0

for i in range(len(df)):
    # 검색 키
    text = df.loc[i,'비고']
    
    ### ('학과' 또는 '학부' 또는 '전공')과 '수강', '가능' 세 단어가 포함된 경우
    condition1 = (any(word in text for word in ['학과', '학부', '전공'])
                  and '수강' in text
                  and '가능' in text)
    ### 다/부전공과 관한 단어가 포함된 경우
    condition2 = (any(word in text for word in ['다전', '다(부', '다/부', '부전', '부)전', '부/전']))
    ### 우선수강, 특정 학생만 수강 등 내용이 포함된 경우 (단, 선수과목에 대한 내용이면 제외)
    condition3 = (any(word1 in text for word1 in ['우선수강','만수강'])
                  and not any(word2 in text for word2 in ['선수과목','선이수','이수자만']))
    ### GTEP 여부
    condition4 = ('GTEP' in text)

    if condition1 or condition2 or condition3 or condition4:
        df.loc[i,'수강대상보호'] = 1        

In [249]:
df.수강대상보호.value_counts()

수강대상보호
0    2315
1    1090
Name: count, dtype: int64

In [250]:
# keywords = ['신산업융합학과','K뷰티산업융합학과']

# df['산업융합학과수강제한'] = 0

# def filter_text(raw_text):
#     # 텍스트 내 공백 제거
#     text = raw_text.replace(" ", "")
#     # 키워드 포함 여부 확인
#     return any(keyword in text for keyword in keywords)

# # DataFrame에 두 컬럼에 대해 조건 적용
# df['산업융합학과수강제한'] = df.apply(
#     lambda row: 1 if (filter_text(row['비고']) or filter_text(row['수강신청유의사항'])) else 0, 
#     axis=1
# )

In [251]:
# filtered_rows = df[df['비고'].apply(filter_text)]
# for row in range(len(filtered_rows)):
#     print(f"{filtered_rows.iloc[row, 0]}, {filtered_rows.iloc[row, 1]}, {filtered_rows.iloc[row, 3]}")
#     print('-----------')

In [252]:
# df.산업융합학과수강제한.value_counts()

### 유학생,교환학생 수강제한 여부

In [253]:
keywords = ['유학생','교환학생']

df['유학생교환학생수강제한'] = 0

def filter_text(raw_text):
    # 텍스트 내 공백 제거
    text = raw_text.replace(" ", "")
    # 키워드 포함 여부 확인
    return any(keyword in text for keyword in keywords)

# DataFrame에 두 컬럼에 대해 조건 적용
df['유학생교환학생수강제한'] = df.apply(
    lambda row: 1 if (filter_text(row['비고']) or filter_text(row['수강신청유의사항'])) else 0, 
    axis=1
)
    
# filtered_rows = df[df['비고'].apply(filter_text)]
# for row in range(len(filtered_rows)):
#     print(f"{filtered_rows.iloc[row, 0]}, {filtered_rows.iloc[row, 1]}, {filtered_rows.iloc[row, 3]}")
#     print('-----------')

In [254]:
df.유학생교환학생수강제한.value_counts()

유학생교환학생수강제한
0    3378
1      27
Name: count, dtype: int64

### 1교시 여부, 월/금요일 여부

In [255]:
# 9시에 시작하는 수업
df['수업시간대'] = df['수업시간대'].fillna('')  # NaN을 빈 문자열로 대체
df['1교시여부'] = df['수업시간대'].apply(lambda x: 1 if '09:00' in x else 0)

# 월요일에 여는 수업
df['수업요일'] = df['수업요일'].fillna('')  # NaN을 빈 문자열로 대체
df['월요일여부'] = df['수업요일'].apply(lambda x: 1 if '월' in x else 0)

# 금요일에 여는 수업
df['금요일여부'] = df['수업요일'].apply(lambda x: 1 if '금' in x else 0)

In [256]:
print(df['1교시여부'].value_counts())
print(df['월요일여부'].value_counts())
print(df['금요일여부'].value_counts())

1교시여부
0    2625
1     780
Name: count, dtype: int64
월요일여부
0    2268
1    1137
Name: count, dtype: int64
금요일여부
0    2908
1     497
Name: count, dtype: int64


### 이론강의팀플존재여부

In [257]:
print(df.강의유형.value_counts())

강의유형
이론          2165
이론+실습        897
실험+실습+실기     343
Name: count, dtype: int64


In [258]:
df['이론강의팀플존재여부'] = 0

for i in range(len(df)):
    if df.loc[i,'조별과제존재여부'] == 1 and df.loc[i,'강의유형'] == '이론':
        df.loc[i,'이론강의팀플존재여부'] = 1

In [259]:
df.이론강의팀플존재여부.value_counts()

이론강의팀플존재여부
0    3292
1     113
Name: count, dtype: int64

## 변수 전처리

### 층

In [260]:
df.층.unique()

array(['2', '2, 2', '3', '4, 4', '1, 1', '3, 3', '1', '4', '온라인', '5, 5',
       '3, 3, 3', '5', '5, 1', '1, 2', '3, 1', '7', '3, 4', '2, 1',
       '5, 5, 5', '6, 6', '6', '1, 5', '8, 8', '8', '3, 5', '2, 5',
       '5, 4', '5, 8', '4, 6', '7, 7', '7, 7, 7, 7', '2, 3', '4, 3',
       '4, 5', '1, 4', '1, 3', '1, 7', '1, 6', '4, 4, 4', '3, 2',
       '3, 6, 3, 6', '3, 6', '4, 1', '1, 3, 3', '3, 3, 1', '2, 3, 3', '9',
       '6, 6, 6', '5, 3', '1, 3, 1', '7, 4', '7, 2', '4, 2', '6, 4',
       '5, 2', '3, 8', '7, 8', '1, 2, 2', '2, 8', '6, 5', '7, 1', '6, 1',
       '1, 9'], dtype=object)

In [261]:
df['층'] = df['층'].apply(lambda text: max(map(int, text.split(','))) if text != '온라인' else 0)

In [262]:
df['층'].value_counts()

층
3    876
2    857
1    552
4    450
5    327
7    128
8    110
6     88
0     15
9      2
Name: count, dtype: int64

In [263]:
df[df['층']==9][['연도','과목번호','교과목명','건물명']]

,연도,과목번호,교과목명,건물명
1080,2022,3344,유기화학실험,신공학관
3330,2024,3319,고분자재료실험,"C공학관, 신공학관"


## 최종

In [264]:
df.columns

Index(['연도', '과목번호', '교과목명', '학점', '시간', '개설학년', '개설학과', '강의종류', '강의유형',
       '전체학년 제한인원', '전체학년 수강바구니', '전체학년 경쟁률', '4학년 제한인원', '4학년 수강바구니',
       '4학년 경쟁률', '3학년 제한인원', '3학년 수강바구니', '3학년 경쟁률', '2학년 제한인원', '2학년 수강바구니',
       '2학년 경쟁률', '1학년 제한인원', '1학년 수강바구니', '1학년 경쟁률', '수업요일', '건물명',
       '일주일 중 수업횟수', '수업시간대', '층', '분반 개수', '패스과목', '원어강의', '절대평가 유무',
       '점심시간 포함 개수', '고학년', '계열', '1학년 남학생 수', '1학년 여학생 수', '1학년 총 학생 수',
       '2학년 남학생 수', '2학년 여학생 수', '2학년 총 학생 수', '3학년 남학생 수', '3학년 여학생 수',
       '3학년 총 학생 수', '4학년 남학생 수', '4학년 여학생 수', '4학년 총 학생 수', '학과 총 학생 수',
       '학과 총 남학생 수', '학과 총 여학생 수', '1학년 남녀 비율', '2학년 남녀 비율', '3학년 남녀 비율',
       '4학년 남녀 비율', '학과 남녀 비율', '개설학년 경쟁률', '개설학년 수강바구니', '개설학년 제한인원', '단과대학',
       '비고', '수강신청유의사항', '평가비율', '교재', '과제내용', '과제수', '주별강의계획서', '중간고사',
       '기말고사', '담당교수', '학수번호', '캡스톤', 'NCS', '선수과목존재여부', '조별과제존재여부',
       '지필고사대체여부', '수강대상보호', '유학생교환학생수강제한', '1교시여부', '월요일여부', '금요일여부',
       '이론강의팀플존재여부'],
      dtype='object')

In [265]:
df.shape

(3405, 82)

In [266]:
df.to_csv('전선_통합_1202.csv',index=False,encoding='cp949')

불필요한 컬럼은 제거합니다.

In [273]:
light_df = df.drop(columns=['수업요일','건물명','수업시간대',
                            '비고','수강신청유의사항','평가비율','교재','과제내용','주별강의계획서','중간고사','기말고사'])

In [274]:
light_df.shape

(3405, 71)

In [275]:
sum(list(light_df.isnull().sum()))

0

In [276]:
light_df.to_csv('전선_실사용_1202.csv',index=False,encoding='cp949')